# SSII - Práctica Final: Recomendador de películas con MovieLens

## Grupo 1: Ignacio Gil, Álvaro Farreny y Carlos González

In [ ]:
""" 
pip install pandas
pip install numpy
python -m pip install -U matplotlib
pip install seaborn
python -m pip install requests
pip install -U scikit-learn   
pip install beautifulsoup4
"""

import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline
import seaborn as sns
import requests as rq
import time
import math
import datetime

from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.metrics import confusion_matrix

from bs4 import BeautifulSoup as bs

In [ ]:
# Leemos nuestros dataframes
movies = pd.read_csv("./ml-latest-small/movies.csv", sep=",")
ratings = pd.read_csv("./ml-latest-small/ratings.csv", sep=",")
tags = pd.read_csv("./ml-latest-small/tags.csv", sep=",")
links = pd.read_csv("./ml-latest-small/links.csv", sep=",")

# Eliminamos nuelos
movies.dropna(inplace=True)
ratings.dropna(inplace=True)
tags.dropna(inplace=True)
links.dropna(inplace=True)

In [ ]:
# Extraemos el año del título
movies['year'] = movies.title.str.extract("\((\d{4})\)", expand=True)
movies.year = pd.to_datetime(movies.year, format='%Y')
movies.year = movies.year.dt.year
movies.title = movies.title.str[:-7]

In [ ]:
'''# Separamos los géneros
movies['genres'] = movies['genres'].str.split('|')
dfx = pd.get_dummies(pd.DataFrame(movies['genres'].tolist()).stack()).sum(level=0)
movies = pd.concat([movies, dfx], axis=1).drop(columns=['genres'])

generos = movies[['movieId', '(no genres listed)', 'Action', 'Adventure',
       'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']].copy()'''

In [ ]:
movies.head()

In [ ]:
# Cambiamos los timestamps de ratings y tags de segundos a años
ratings.timestamp = pd.to_datetime(ratings.timestamp, infer_datetime_format=True)
ratings.timestamp = ratings.timestamp.dt.year

tags.timestamp = pd.to_datetime(tags.timestamp, infer_datetime_format=True)
tags.timestamp = tags.timestamp.dt.year

In [ ]:
ratings.head()

In [ ]:
ratings.info()

In [ ]:
tags.head()

In [ ]:
links.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations

# TF-IDF para las diferentes combinaciones de géneros
tf = TfidfVectorizer(analyzer=lambda s: (c for i in range(1,4)
                     for c in combinations(s.split('|'), r=i)))

# Formamos nuestra matriz con los géneros
matriz = tf.fit_transform(movies.genres)
# Aplicamos a esta matriz la similitud del coseno
similitud = cosine_similarity(matriz)
# Creamos un DF con esta similitud
similitud_df = pd.DataFrame(similitud, index=movies['title'], columns=movies['title'])

In [ ]:
similitud_df

In [ ]:
similitud_df.shape

In [ ]:
movies[['title', 'genres']]

In [ ]:
# Nuestro valor k indicará la cantidad de recomendaciones que damos
def genre_recommendations(i, M, items, k=10):
    """
    i : str
        Movie (index of the similarity dataframe)
    M : pd.DataFrame
        Similarity dataframe, symmetric, with movies as indices and columns
    items : pd.DataFrame
        Contains both the title and some other features used to define similarity
    k : int
        Amount of recommendations to return
    """
    index = M.loc[:,i].to_numpy().argpartition(range(-1,-k,-1))
    closest = M.columns[index[-1:-(k+2):-1]]
    closest = closest.drop(i, errors='ignore')
    return pd.DataFrame(closest).merge(items).head(k)

In [ ]:
genre_recommendations('Toy Story', similitud_df, movies[['title', 'genres']])

In [ ]:
similitud_df.to_csv("prueba1.csv")

In [1]:
""" 
pip install pandas
pip install numpy
python -m pip install -U matplotlib
pip install seaborn
python -m pip install requests
pip install -U scikit-learn   
pip install beautifulsoup4
"""

import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline
import seaborn as sns
import requests as rq
import time
import math
import datetime

from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.metrics import confusion_matrix

from bs4 import BeautifulSoup as bs

# Leemos nuestros dataframes
movies = pd.read_csv("./ml-latest-small/movies.csv", sep=",")
ratings = pd.read_csv("./ml-latest-small/ratings.csv", sep=",")
tags = pd.read_csv("./ml-latest-small/tags.csv", sep=",")
links = pd.read_csv("./ml-latest-small/links.csv", sep=",")

# Eliminamos nuelos
movies.dropna(inplace=True)
ratings.dropna(inplace=True)
tags.dropna(inplace=True)
links.dropna(inplace=True)

# Extraemos el año del título
movies['year'] = movies.title.str.extract("\((\d{4})\)", expand=True)
movies.year = pd.to_datetime(movies.year, format='%Y')
movies.year = movies.year.dt.year
movies.title = movies.title.str[:-7]

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations

# TF-IDF para las diferentes combinaciones de géneros
tf = TfidfVectorizer(analyzer=lambda s: (c for i in range(1,4)
                     for c in combinations(s.split('|'), r=i)))

# Formamos nuestra matriz con los géneros
matriz = tf.fit_transform(movies.genres)
# Aplicamos a esta matriz la similitud del coseno
similitud = cosine_similarity(matriz)
# Creamos un DF con esta similitud
similitud_df = pd.DataFrame(similitud, index=movies['title'], columns=movies['title'])

# Nuestro valor k indicará la cantidad de recomendaciones que damos
def genre_recommendations(i, M, items, k=10):
    """
    i : str
        Movie (index of the similarity dataframe)
    M : pd.DataFrame
        Similarity dataframe, symmetric, with movies as indices and columns
    items : pd.DataFrame
        Contains both the title and some other features used to define similarity
    k : int
        Amount of recommendations to return
    """
    index = M.loc[:,i].to_numpy().argpartition(range(-1,-k,-1))
    closest = M.columns[index[-1:-(k+2):-1]]
    closest = closest.drop(i, errors='ignore')
    return pd.DataFrame(closest).merge(items).head(k)

genre_recommendations('Toy Story', similitud_df, movies[['title', 'genres']])

,title,genres
0,"Adventures of Rocky and Bullwinkle, The",Adventure|Animation|Children|Comedy|Fantasy
1,"Emperor's New Groove, The",Adventure|Animation|Children|Comedy|Fantasy
2,Toy Story 2,Adventure|Animation|Children|Comedy|Fantasy
3,Antz,Adventure|Animation|Children|Comedy|Fantasy
4,Turbo,Adventure|Animation|Children|Comedy|Fantasy
5,"Tale of Despereaux, The",Adventure|Animation|Children|Comedy|Fantasy
6,"Monsters, Inc.",Adventure|Animation|Children|Comedy|Fantasy
7,Shrek the Third,Adventure|Animation|Children|Comedy|Fantasy
8,The Good Dinosaur,Adventure|Animation|Children|Comedy|Fantasy
9,Asterix & Obelix vs. Caesar (Astérix et Obélix...,Adventure|Children|Comedy|Fantasy


## Obtener Similitudes a través de TF-IDF y Similitud del coseno

In [ ]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations

# Leemos nuestros dataframes
movies = pd.read_csv("./ml-latest-small/movies.csv", sep=",")
sinopsis = pd.read_csv("./ml-latest-small/sinopsisDB.csv", sep=",")
sinopsis = sinopsis.sort_values(by='movieId')

In [ ]:
# Partimos sinopsis en 5
partes = np.array_split(sinopsis, 5)
sinopsis1 = partes[0]
sinopsis2 = partes[1]
sinopsis3 = partes[2]
sinopsis4 = partes[3]
sinopsis5 = partes[4]

# Exportamos estas sinopsis
sinopsis1.to_csv('sinopsis1.csv', index=False)
sinopsis2.to_csv('sinopsis2.csv', index=False)
sinopsis3.to_csv('sinopsis3.csv', index=False)
sinopsis4.to_csv('sinopsis4.csv', index=False)
sinopsis5.to_csv('sinopsis5.csv', index=False)

# Extraemos el año del título
movies['year'] = movies.title.str.extract("\((\d{4})\)", expand=True)
movies.year = pd.to_datetime(movies.year, format='%Y')
movies.year = movies.year.dt.year
movies.title = movies.title.str[:-7]

In [ ]:
# TF-IDF para las diferentes combinaciones de géneros
tfG = TfidfVectorizer(analyzer=lambda s: (c for i in range(1,4)
                     for c in combinations(s.split('|'), r=i)))
tfS = TfidfVectorizer(analyzer=lambda s: (c for i in range(1,4)
                     for c in combinations(s.split(' '), r=i)))

# Formamos nuestra matriz con los géneros
matrizG = tfG.fit_transform(movies.genres)
matrizS1 = tfS.fit_transform(sinopsis1.sinopsis)
# Aplicamos a esta matriz la similitud del coseno
similitudG = cosine_similarity(matrizG)
similitudS1 = cosine_similarity(matrizS1)

pd.DataFrame(similitudG).to_csv("similitudG.csv", header=False, index=False)
pd.DataFrame(similitudS1).to_csv("similitudS1.csv", header=False, index=False)

In [ ]:
matrizS2 = tfS.fit_transform(sinopsis2.sinopsis)
similitudS2 = cosine_similarity(matrizS2)
pd.DataFrame(similitudS2).to_csv("similitudS2.csv", header=False, index=False)

In [ ]:
matrizS3 = tfS.fit_transform(sinopsis3.sinopsis)
similitudS3 = cosine_similarity(matrizS3)
pd.DataFrame(similitudS3).to_csv("similitudS3.csv", header=False, index=False)

In [ ]:
matrizS4 = tfS.fit_transform(sinopsis4.sinopsis)
similitudS4 = cosine_similarity(matrizS4)
pd.DataFrame(similitudS4).to_csv("similitudS4.csv", header=False, index=False)

In [ ]:
matrizS5 = tfS.fit_transform(sinopsis5.sinopsis)
similitudS5 = cosine_similarity(matrizS5)
pd.DataFrame(similitudS5).to_csv("similitudS5.csv", header=False, index=False)

In [ ]:
# Cargamos similitudes
similitudG = pd.read_csv("./ml-latest-small/similitudG.csv", sep=",")
similitudS1 = pd.read_csv("./ml-latest-small/similitudS1.csv", sep=",")
similitudS2 = pd.read_csv("./ml-latest-small/similitudS2.csv", sep=",")
similitudS3 = pd.read_csv("./ml-latest-small/similitudS3.csv", sep=",")
similitudS4 = pd.read_csv("./ml-latest-small/similitudS4.csv", sep=",")
similitudS5 = pd.read_csv("./ml-latest-small/similitudS5.csv", sep=",")

# Creamos un DF con cada similitud
similitud_dfG = pd.DataFrame(similitudG, index=movies['title'], columns=movies['title'])
similitud_dfS1 = pd.DataFrame(similitudS1, index=movies['title'], columns=movies['title'])
similitud_dfS2 = pd.DataFrame(similitudS2, index=movies['title'], columns=movies['title'])
similitud_dfS3 = pd.DataFrame(similitudS3, index=movies['title'], columns=movies['title'])
similitud_dfS4 = pd.DataFrame(similitudS4, index=movies['title'], columns=movies['title'])
similitud_dfS5 = pd.DataFrame(similitudS5, index=movies['title'], columns=movies['title'])

similitud_dfG.to_csv("./ml-latest-small/similitud_dfG.csv", header=False, index=False)
similitud_dfS1.to_csv("./ml-latest-small/similitud_dfS1.csv", header=False, index=False)
similitud_dfS2.to_csv("./ml-latest-small/similitud_dfS2.csv", header=False, index=False)
similitud_dfS3.to_csv("./ml-latest-small/similitud_dfS3.csv", header=False, index=False)
similitud_dfS4.to_csv("./ml-latest-small/similitud_dfS4.csv", header=False, index=False)
similitud_dfS5.to_csv("./ml-latest-small/similitud_dfS5.csv", header=False, index=False)